[Reference](https://medium.com/better-programming/how-to-send-emails-with-attachments-using-python-dd37c4b6a7fd)

# Secure connection

In [4]:
import smtplib
import ssl

context = ssl.create_default_context()

with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
    smtp.noop()

In [7]:
import smtplib
import ssl

SERVER_ADDRESS = "YOUR_SERVER_ADDRESS"  # smtp.live.com for example
SERVER_PORT = 587
EMAIL_ADDRESS = 'YOUR_EMAIL_ADDRESS@EXAMPLE_DOMAIN.COM'
EMAIL_PASSWORD = 'YOUR_PASSWORD'
RECIPIENT_EMAIL = 'RECIPIENT_EMAIL@EXAMPLE_DOMAIN.COM'

# Email content
email_subject = "My Custom Subject"
email_sender = EMAIL_ADDRESS
email_recipient = RECIPIENT_EMAIL

message = f"""\
Subject: {email_subject}
From: {email_sender}
TO: {email_recipient}
Hello World"""

context = ssl.create_default_context()

with smtplib.SMTP(SERVER_ADDRESS, SERVER_PORT) as smtp:
    smtp.ehlo()  # Say EHLO to server
    smtp.starttls(context=context)  # Puts the connection in TLS mode.
    smtp.ehlo()
    smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
    smtp.sendmail(EMAIL_ADDRESS, RECIPIENT_EMAIL, message)

# Send formatted HTML text

In [8]:
import smtplib
import ssl
from email.message import EmailMessage


SERVER_ADDRESS = "YOUR_SERVER_ADDRESS"  # smtp.live.com for example
SERVER_PORT = 587
EMAIL_ADDRESS = 'YOUR_EMAIL_ADDRESS@EXAMPLE_DOMAIN.COM'
EMAIL_PASSWORD = 'YOUR_PASSWORD'
RECIPIENT_EMAIL = 'RECIPIENT_EMAIL@EXAMPLE_DOMAIN.COM'

# Email content
msg = EmailMessage()

msg['Subject'] = "My Custom Subject"
msg['From'] = EMAIL_ADDRESS
msg['To'] = RECIPIENT_EMAIL

msg.set_content('Hello World')

msg.add_alternative("""
<p>
    <h1>My Custom Title</h1>
    Hello <strong>World</strong>
</p>
""", subtype='html')


# Create a SSLContext object with default settings.
context = ssl.create_default_context()

with smtplib.SMTP(SERVER_ADDRESS, SERVER_PORT) as smtp:
    smtp.ehlo()  # Say EHLO to server
    smtp.starttls(context=context)  # Puts the connection in TLS mode.
    smtp.ehlo()
    smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
    smtp.send_message(msg)  # Auto detects the sender and recipient from header

# Send attachments

In [9]:
import mimetypes
import smtplib
import ssl
from email.message import EmailMessage


SERVER_ADDRESS = "YOUR_SERVER_ADDRESS"  # smtp.live.com for example
SERVER_PORT = 587
EMAIL_ADDRESS = 'YOUR_EMAIL_ADDRESS@EXAMPLE_DOMAIN.COM'
EMAIL_PASSWORD = 'YOUR_PASSWORD'
RECIPIENT_EMAIL = 'RECIPIENT_EMAIL@EXAMPLE_DOMAIN.COM'

# Email content
msg = EmailMessage()

msg['Subject'] = "My Custom Subject"
msg['From'] = EMAIL_ADDRESS
msg['To'] = RECIPIENT_EMAIL

msg.set_content('Hello World')

msg.add_alternative("""
<p>
    <h1>My Custom Title</h1>
    Hello <strong>World</strong>
</p>
""", subtype='html')


filename = 'attachment.txt'
path = f'docs/{filename}'

# Guess the content type based on the file's extension.
ctype, encoding = mimetypes.guess_type(path)
if ctype is None or encoding is not None:
    ctype = 'application/octet-stream'
maintype, subtype = ctype.split('/', 1)

with open(path, 'rb') as fp:
    msg.add_attachment(fp.read(), maintype=maintype, subtype=subtype,
                       filename=filename)


# Create a SSLContext object with default settings.
context = ssl.create_default_context()

with smtplib.SMTP(SERVER_ADDRESS, SERVER_PORT) as smtp:
    smtp.ehlo()  # Say EHLO to server
    smtp.starttls(context=context)  # Puts the connection in TLS mode.
    smtp.ehlo()
    smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
    smtp.send_message(msg)  # Auto detects the sender and recipient from header

# How to Send Markdown Templates Using Dmail

In [10]:
!pip install Dmail

     |████████████████████████████████| 409kB 7.5MB/s 


In [11]:
import traceback
from Dmail.esp import Hotmail


EMAIL_ADDRESS = 'YOUR_EMAIL_ADDRESS@EXAMPLE_DOMAIN.COM'
EMAIL_PASSWORD = 'YOUR_PASSWORD'
RECIPIENT_EMAIL = 'RECIPIENT_EMAIL@EXAMPLE_DOMAIN.COM'


try:
    raise Exception('Something went wrong')
except Exception as e:
    # Email Content
    msg = '\n'.join(['# Traceback',
                     '```pytb',
                     traceback.format_exc(),
                     '```'])
    # Sending the email
    with Hotmail(EMAIL_ADDRESS, EMAIL_PASSWORD) as email:
        email.send(msg, RECIPIENT_EMAIL, subject=f'Failed job: {e}')

In [12]:
import pandas as pd
from Dmail.esp import Hotmail


# Adding personal information
recipient_name = 'USER_NAME'
sender_name = 'SENDER_NAME'
email_address = 'YOUR_EMAIL_ADDRESS@EXAMPLE_DOMAIN.COM'
password = 'YOUR_PASSWORD'
recipient_email = 'RECIPIENT_EMAIL@EXAMPLE_DOMAIN.COM'


# Email template creation
template = """\
Dear {recipient},
This is just an email example containing:
- The **banner image** as an inline image
- The **SMTP addresses table**, with *centered cell values*, some *background colors* and *automatically adjusted 
height* 
- A csv **file attached**
Here's the table:
{smtp_table}
You can also find below the article banner:
![Article Banner Image]({image_path})
Best regards,
{sender}
"""

# Import the table from the csv file in a pandas dataframe
table_path = "docs/smtp.csv"
smtp_table = pd.read_csv(table_path, sep=";", index_col=None)

# Get image path
image_path = "docs/banner.png"


# Function that highlights odd rows with specific color in a pandas dataframe
def highlight_odd_rows(s):
    return ['background-color: #CEF8BE' if s.name % 2 else '' for _ in s]


# Styling the dataframe using pandas' .style
smtp_table = (smtp_table.style
              # .set_caption("SMTP addresses table")      # Add caption to table
              .set_properties(**{'text-align': 'center',  # Align cell values to center
                                 'margin': 'auto'})  # Adjust cell sizes automatically
              .set_table_styles([{'selector': 'th',
                                  'props': [('background-color', '#6BE63E'),  # Add background color to header
                                            ('margin', 'auto')]}])  # Adjust header cell sizes automatically
              .apply(highlight_odd_rows, axis=1)  # Add background color to odd rows
              .hide_index()  # Export the table without the index column
              .render())


# Creating the email body
message = template.format(recipient=recipient_name, sender=sender_name, image_path=image_path, smtp_table=smtp_table)

# Sending the email
with Hotmail(email_address, password) as email:
    email.send(message, recipient_email, attachments=[table_path],
               subject=f"Demonstration of mail sending using Dmail library")